# LanguageTool

In [15]:
#!pip install language_tool_python

In [16]:
import language_tool_python
tool = language_tool_python.LanguageTool('fr')

In [17]:
text = 'Paire de skis quasi neuf en très bonne étot datant de l année dernière'
matches = tool.check(text)
print(len(matches))
matches[0].ruleId, matches[0].replacements

2


('FR_SPELLING_RULE',
 ['état',
  'tôt',
  'étau',
  'égout',
  'écot',
  'étaux',
  'ethos',
  'stot',
  'éthos',
  'étoc'])

In [178]:
print(matches[0])

Offset 39, length 4, Rule ID: FR_SPELLING_RULE
Message: Faute de frappe possible trouvée.
Suggestion: état; tôt; étau; égout; écot; étaux; ethos; stot; éthos; étoc
Paire de skis quasi neuf en très bonne étot davant l année dernière pas
                                       ^^^^


In [179]:
tool.correct(text)

'Paire de skis quasi neuf en très bonne état avant l’année dernière pas'

## Grammalecte

In [18]:
#!pip install pygrammalecte

In [2]:
import pygrammalecte
import unicodedata
import language_tool_python

text = """Paire de Skis quasi neufs en très bonne état, modèle de lannée dernière ac fixatins Salomon
très bonne affaire n hésitez pas !!!"""

In [20]:
print(pygrammalecte.correct(text))

Paire de Skis quasi neufs en très bon état, modèle de l'annexe dernière avec fixations Salomon.
très bonne affaire n'hésitez pas !


* Development

In [3]:
from pygrammalecte import grammalecte_text

for message in grammalecte_text(text):
    print(message)

Ligne 1 [40:44] [g2__gn_pfx_en_2m__b1_a1_1] Accord de genre erroné avec « bonne ».
Ligne 1 [34:39] [g2__gn_pfx_en_2m__b1_a2_1] Accord de genre erroné avec « état ». (Suggestions : bon)
Ligne 1 [56:62] Mot inconnu : lannée
Ligne 1 [75:83] Mot inconnu : fixatins
Ligne 2 [32:34] [nbsp_avant_double_ponctuation] Il manque un espace insécable. (Suggestions :  !)


In [6]:
other_corrections = {
    "!!": "!",
    "??": "?",
    "?!": "?",
    " d ": " d'",
    " n ": " n'",
    " l ": " l'",
    "tps": "temps",
    "ac": "avec",
    "ok": "OK",
    "okay": "OK",
    "pr": "pour",
}

def replace_values_in_string(text, args_dict):
    for key in args_dict.keys():
        text = text.replace(key, str(args_dict[key]))
    return text

def correct(text: str) -> str:
    
    # get correction messages and sort them in reverse order
    messages = list(pygrammalecte.grammalecte_text(text, as_dict=True))
    messages = sorted(messages, key = lambda i: (i['line'], i['start']), reverse=True)
    
    # split text in lines and characters
    lines = text.splitlines()
    lines = [list(x) for x in lines]
    
    # apply corrections
    unknown_words = []
    for message in messages:
        line = lines[message['line']-1]
        if 'word' in message:
            unknown_words.append(message['word']) # store unknown words for later
        try:
            line[message['start']:message['end']] = list(message['suggestions'][0])
        except:
            continue
    
    # rebuild text
    corrected_lines = [''.join(x) for x in lines]
    corrected_lines = [l + '.' if not l.endswith(('.', '!', '?', ',', ';', ':')) else l for l in corrected_lines]
    corrected_lines = [l[0].upper() + l[1:] if l[0].upper() != l[0] else l for l in corrected_lines]
    corrected_text = '\n'.join(corrected_lines)
    
    # perform additional checks
    if any(x in corrected_text for x in other_corrections):
        corrected_text = replace_values_in_string(corrected_text, other_corrections)
    for word in unknown_words:
        if word.startswith(('n', 'l', 'd')):
            new_word = word[:1] + "'" + word[1:]
            corrected_text = corrected_text.replace(word, new_word)
    
    # perform languagetool corrections on top of everything
    tool = language_tool_python.LanguageTool('fr')
    corrected_text = tool.correct(corrected_text)
    corrected_text = unicodedata.normalize("NFKD", corrected_text)
    
    return corrected_text

In [7]:
corrected_text = correct(text)
print(corrected_text)

Paire de Skis quasi neufs en très bon état, modèle de l'année dernière avec fixations Salomon.
Très bonne affaire n'hésitez pas !!
